# 03 - Aggregation Strategies Overview

This notebook provides a concise guide to the main aggregation strategies in the federated learning framework.

## What You'll Learn
- Key aggregation strategies and their purposes
- How to configure and use different strategies
- Performance comparison and selection guide
- Best practices for production use

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

from core.strategies import create_strategy
import numpy as np
import torch
import flwr as fl
import matplotlib.pyplot as plt

## 2. Strategy Categories

The framework supports three main categories of aggregation strategies:

In [ ]:
# Strategy overview
strategies = {
    'Basic Strategies': {
        'FedAvg': 'Standard weighted averaging - most common',
        'FedAvgM': 'FedAvg with server momentum',
        'FedAdam': 'Adaptive optimization with Adam'
    },
    'Robust Strategies': {
        'Krum': 'Byzantine-tolerant, selects trustworthy updates',
        'TrimmedMean': 'Removes outliers before averaging',
        'Bulyan': 'Multi-Krum with additional robustness'
    },
    'Advanced Strategies': {
        'SCAFFOLD': 'Variance reduction for non-IID data',
        'FedProx': 'Proximal regularization for heterogeneity',
        'FedNova': 'Normalized averaging for unequal local steps'
    }
}

for category, strats in strategies.items():
    print(f"\n{category.upper()}:")
    for name, desc in strats.items():
        print(f"  • {name}: {desc}")

## 3. Creating and Configuring Strategies

In [ ]:
# Basic FedAvg - most common choice
fedavg_strategy = create_strategy('fedavg')
print(f"FedAvg: {type(fedavg_strategy).__name__}")

# Robust Krum for adversarial environments
krum_strategy = create_strategy('krum', num_malicious_clients=2)
print(f"Krum: {type(krum_strategy).__name__}")

# FedProx for heterogeneous clients
fedprox_strategy = create_strategy('fedprox', proximal_mu=0.1)
print(f"FedProx: {type(fedprox_strategy).__name__}")

print("\n✅ Strategies created successfully!")

## 4. Strategy Comparison Simulation

Compare strategies under different conditions:

In [ ]:
# Simulate strategy performance under different conditions
def simulate_strategy_performance(strategy_name, condition):
    """Simulate how strategies perform under different conditions."""
    
    # Base performance values (simulated)
    base_performance = {
        'FedAvg': {'iid': 0.92, 'non_iid': 0.78, 'byzantine': 0.45},
        'FedProx': {'iid': 0.90, 'non_iid': 0.85, 'byzantine': 0.50},
        'Krum': {'iid': 0.88, 'non_iid': 0.75, 'byzantine': 0.80},
        'TrimmedMean': {'iid': 0.89, 'non_iid': 0.77, 'byzantine': 0.82},
        'SCAFFOLD': {'iid': 0.93, 'non_iid': 0.88, 'byzantine': 0.48}
    }
    
    return base_performance.get(strategy_name, {}).get(condition, 0.0)

# Test strategies under different conditions
test_strategies = ['FedAvg', 'FedProx', 'Krum', 'TrimmedMean', 'SCAFFOLD']
conditions = ['iid', 'non_iid', 'byzantine']

results = {}
for condition in conditions:
    results[condition] = []
    for strategy in test_strategies:
        perf = simulate_strategy_performance(strategy, condition)
        results[condition].append(perf)

print("Strategy Performance Comparison:")
print("=" * 45)
print(f"{'Strategy':<12} {'IID':<8} {'Non-IID':<8} {'Byzantine':<10}")
print("-" * 45)
for i, strategy in enumerate(test_strategies):
    print(f"{strategy:<12} {results['iid'][i]:<8.2f} {results['non_iid'][i]:<8.2f} {results['byzantine'][i]:<10.2f}")

## 5. Visual Comparison

In [ ]:
# Create performance comparison chart
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Strategy Performance Under Different Conditions', fontsize=16)

for i, condition in enumerate(conditions):
    ax = axes[i]
    colors = ['blue', 'green', 'red', 'orange', 'purple']
    
    bars = ax.bar(test_strategies, results[condition], color=colors, alpha=0.7)
    ax.set_title(f'{condition.upper()} Data')
    ax.set_ylabel('Accuracy')
    ax.set_ylim(0, 1.0)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, results[condition]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 6. Strategy Selection Guide

In [ ]:
# Strategy selection decision tree
def recommend_strategy(data_distribution, has_malicious_clients, client_heterogeneity):
    """
    Recommend the best strategy based on environment characteristics.
    
    Args:
        data_distribution: 'iid' or 'non_iid'
        has_malicious_clients: True/False
        client_heterogeneity: 'low', 'medium', 'high'
    """
    
    if has_malicious_clients:
        return 'Krum or TrimmedMean (Byzantine tolerance needed)'
    elif data_distribution == 'non_iid' and client_heterogeneity == 'high':
        return 'SCAFFOLD or FedProx (handles non-IID + heterogeneity)'
    elif data_distribution == 'non_iid':
        return 'FedProx or SCAFFOLD (non-IID data)'
    elif client_heterogeneity == 'high':
        return 'FedProx (client heterogeneity)'
    else:
        return 'FedAvg (standard choice for IID data)'

# Test different scenarios
scenarios = [
    ('iid', False, 'low'),
    ('non_iid', False, 'medium'),
    ('iid', True, 'low'),
    ('non_iid', True, 'high'),
    ('non_iid', False, 'high')
]

print("STRATEGY SELECTION GUIDE:")
print("=" * 50)

for i, (dist, malicious, hetero) in enumerate(scenarios, 1):
    recommendation = recommend_strategy(dist, malicious, hetero)
    print(f"\nScenario {i}:")
    print(f"  Data: {dist.upper()}")
    print(f"  Malicious clients: {'Yes' if malicious else 'No'}")
    print(f"  Client heterogeneity: {hetero.upper()}")
    print(f"  → Recommended: {recommendation}")

## 7. Key Strategy Parameters

In [ ]:
# Important parameters for each strategy
strategy_params = {
    'FedAvg': {
        'min_fit_clients': 'Minimum clients for training',
        'min_eval_clients': 'Minimum clients for evaluation',
        'fraction_fit': 'Fraction of clients to use per round'
    },
    'FedProx': {
        'proximal_mu': 'Proximal term strength (0.01-1.0)',
        'all FedAvg params': 'Plus standard FedAvg parameters'
    },
    'Krum': {
        'num_malicious_clients': 'Expected number of malicious clients',
        'to_keep': 'Number of clients to select'
    },
    'TrimmedMean': {
        'beta': 'Fraction of outliers to remove (0.1-0.3)'
    },
    'SCAFFOLD': {
        'all FedAvg params': 'Standard parameters',
        'Note': 'Requires client-side control variates'
    }
}

print("KEY PARAMETERS BY STRATEGY:")
print("=" * 40)

for strategy, params in strategy_params.items():
    print(f"\n{strategy.upper()}:")
    for param, desc in params.items():
        print(f"  • {param}: {desc}")

## 8. Running Experiments with Different Strategies

Example of how to run experiments comparing strategies:

In [ ]:
# Example experiment configuration
experiment_config = {
    'dataset': 'MNIST',
    'num_clients': 10,
    'num_rounds': 20,
    'strategies_to_compare': ['FedAvg', 'FedProx', 'Krum']
}

print("To run strategy comparison experiments:")
print("")
print("from experiment_runners.enhanced_experiment_runner import main")
print("")
print("# Configure strategies to compare")
print("strategies = ['FedAvg', 'FedProx', 'Krum']")
print("")
print("# Run comparison")
print("for strategy in strategies:")
print("    config = {")
for key, value in experiment_config.items():
    if key != 'strategies_to_compare':
        print(f"        '{key}': {repr(value)},")
print("        'strategy': strategy")
print("    }")
print("    # results = main(config)")
print("")
print("⚠️  Real experiments may take significant time to complete.")

## Summary

This notebook covered the essential aggregation strategies:

✅ **Basic Strategies**: FedAvg for standard scenarios, FedAvgM and FedAdam for enhanced optimization

✅ **Robust Strategies**: Krum, TrimmedMean, and Bulyan for Byzantine-tolerant aggregation

✅ **Advanced Strategies**: SCAFFOLD for non-IID data, FedProx for heterogeneous clients, FedNova for unequal local updates

✅ **Selection Guide**: How to choose the right strategy based on your environment

### Quick Decision Guide:
- **Standard case**: Use FedAvg
- **Non-IID data**: Use FedProx or SCAFFOLD
- **Malicious clients**: Use Krum or TrimmedMean
- **Heterogeneous clients**: Use FedProx
- **Research/comparison**: Test multiple strategies

### Next Steps:
- Run comparative experiments with your specific dataset
- Tune hyperparameters for optimal performance
- Consider combining strategies with attack scenarios (see notebook 04)